In [ ]:
import glob
import SimpleITK as sitk
import numpy as np
import collections
from PIL import Image, ImageDraw
#from segmentation import segment
import os
import csv
import ast
import collections
from collections import namedtuple
import random
from pylidc_func import create_ann_malcsv, create_ann_maldf
#from cmask_dict import cmask_dict

isLungSegmentProcess = False

XyzTuple = collections.namedtuple('XyzTuple', ['x', 'y', 'z'])
IrcTuple = collections.namedtuple('IrcTuple', ['index', 'row', 'col'])
def xyz2irc(coord_xyz, origin_xyz, vxSize_xyz, direction_a):
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz)
    coord_a = np.array(coord_xyz)
    cri_a = ((coord_a - origin_a) @ np.linalg.inv(direction_a)) / vxSize_a
    cri_a = np.round(cri_a)
    return IrcTuple(int(cri_a[2]), int(cri_a[1]), int(cri_a[0]))

# df_mal = create_ann_maldf()
# create_ann_malcsv(df_mal)
#print(cmask_dict)
# import json 

# with open('cmask_dict.py', 'w') as file:
#     file.write(str("dict = ").rstrip('\n'))
# 	file.write(str(cmask_dict))

CandidateInfoTuple = namedtuple('CandidateInfoTuple', 'isNodule_bool, hasAnnotation_bool, isMal_bool, series_uid, center_xyz, cbbox, mask_box')
def getCandidateInfoList(requireOnDisk_bool=True):
    # We construct a set with all series_uids that are present on disk.
    # This will let us use the data, even if we haven't downloaded all of
    # the subsets yet.
    mhd_list = glob.glob('D:/LUNA/Luna16_AugData/subset*/*.mhd')
    presentOnDisk_set = {os.path.split(p)[-1][:-4] for p in mhd_list}

    candidateInfo_list = []
    with open('./annotations_object_detection_1127.csv', "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]
            annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
            # annotationDiameter_mm = float(row[4])
            isMal_bool = {'0.0': False, '1.0': True}[row[5]] #it record the malignancy or not
            cbbox = ast.literal_eval(row[7])
            mask_box = row[8]
            mask_list = ast.literal_eval(mask_box)
            
            if series_uid not in presentOnDisk_set and requireOnDisk_bool:
                continue

            candidateInfo_list.append(
                CandidateInfoTuple(
                    True,
                    True,
                    isMal_bool,
                    series_uid,
                    annotationCenter_xyz,
                    cbbox,
                    mask_list
                )
            )
    candidateInfo_list.sort(reverse=True)
    return candidateInfo_list

candidateInfo_list = getCandidateInfoList()
len(candidateInfo_list)

candidateInfo_list[0]

mal_count_list = {'Benign': 0, 'Malignancy': 0, 'nan': 0}
for candidate in candidateInfo_list:
    if candidate[2]:
        for item in candidate[6]:
            if item == 'nan':
                mal_count_list['nan'] += 1
            else:
                mal_count_list['Malignancy'] += 1
    else:
         for item in candidate[6]:
            if item == 'nan':
                mal_count_list['nan'] += 1
            else:
                mal_count_list['Benign'] += 1

print(mal_count_list)

generate = {'Train': {'Benign': mal_count_list['Benign'] - 2 * int(mal_count_list['Benign'] * 0.1), 'Malignancy': mal_count_list['Malignancy'] - 2 * int(mal_count_list['Malignancy'] * 0.1)}, 'Valid': {'Benign':  int(mal_count_list['Benign'] * 0.1), 'Malignancy': int(mal_count_list['Malignancy'] * 0.1)}, 'Test': {'Benign':  int(mal_count_list['Benign'] * 0.1), 'Malignancy':  int(mal_count_list['Malignancy'] * 0.1)}}
print(generate)
# def buildAnnotationMaskNew(noduleInfo_list, mask):
#     for candidateInfo_tup in noduleInfo_list:
#         bbox_info = 
        
        
def buildAnnotationMask(noduleInfo_list, hu_a, origin_xyz, vxSize_xyz, direction_a, threshold_hu = -700):
        boundingBox_a = np.zeros_like(hu_a, dtype=bool) # all False tensor
        bbox_result = []
        for candidateInfo_tup in noduleInfo_list: #traverse所有的nodules
            center_irc = xyz2irc(
                candidateInfo_tup.center_xyz,
                origin_xyz,
                vxSize_xyz,
                direction_a,
            )
            ci = int(center_irc.index)  # the center of voxel
            cr = int(center_irc.row)
            cc = int(center_irc.col)

            index_radius = 2
            try:
                # 從index找，看哪裡會遇到空氣，當兩邊都遇到空氣後就把邊界設在大的那段
                while hu_a[ci + index_radius, cr, cc] > threshold_hu and \
                        hu_a[ci - index_radius, cr, cc] > threshold_hu:
                    index_radius += 1
            except IndexError:
                index_radius -= 1

            row_radius = 2
            try:
                # 從row找，看哪裡會遇到空氣，當兩邊都遇到空氣後就把邊界設在大的那段
                while hu_a[ci, cr + row_radius, cc] > threshold_hu and \
                        hu_a[ci, cr - row_radius, cc] > threshold_hu:
                    row_radius += 1
            except IndexError:
                row_radius -= 1

            col_radius = 2
            try:
                # 從column找，看哪裡會遇到空氣，當兩邊都遇到空氣後就把邊界設在大的那段
                while hu_a[ci, cr, cc + col_radius] > threshold_hu and \
                        hu_a[ci, cr, cc - col_radius] > threshold_hu:
                    col_radius += 1
            except IndexError:
                col_radius -= 1

            # assert index_radius > 0, repr([candidateInfo_tup.center_xyz, center_irc, self.hu_a[ci, cr, cc]])
            # assert row_radius > 0
            # assert col_radius > 0

            boundingBox_a[
                 ci - index_radius: ci + index_radius + 1,
                 cr - row_radius: cr + row_radius + 1,
                 cc - col_radius: cc + col_radius + 1] = True #將box裡的所有格子設成TRUE
            
            bbox_result.append([round((abs(cr) + 1) / 512, 10), round((abs(cc) + 1) / 512, 10), round(2 * row_radius / 512, 10), round(2 * col_radius / 512, 10)])

        #mask_a = boundingBox_a & (hu_a > threshold_hu)  #最後會對box和threshold低於-700的值做and
        mask_a = boundingBox_a
        return mask_a, bbox_result

def getCandidateInfoDict(requireOnDisk_bool=True):  #把candidateInfoList包成Dict
    candidateInfo_list = getCandidateInfoList(requireOnDisk_bool)
    candidateInfo_dict = {}

    for candidateInfo_tup in candidateInfo_list:
        candidateInfo_dict.setdefault(candidateInfo_tup.series_uid,
                                      []).append(candidateInfo_tup)

    return candidateInfo_dict

candidate_dict =  getCandidateInfoDict()
len(candidate_dict)

print(candidate_dict)
def np2Png(np_arr, target_path):
    min_value = np.min(np_arr)
    max_value = np.max(np_arr) if np.max(np_arr) > 0 else 1
    scaled_np_arr = (np_arr - min_value) / (max_value - min_value) * 255
    scaled_np_arr = scaled_np_arr.astype(np.uint8)
    slices = Image.fromarray(scaled_np_arr, mode='L')
    slices = slices.resize((640, 640))
    slices.save(target_path, cmap='gray')

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image 
from matplotlib.patches import Rectangle

def np2MarkPng(np_arr, target_path, row, col):
    min_value = np.min(np_arr)
    max_value = np.max(np_arr)
    scaled_np_arr = (np_arr - min_value) / (max_value - min_value) * 255
    scaled_np_arr = scaled_np_arr.astype(np.uint8)
    slices = Image.fromarray(scaled_np_arr, mode='L')
    draw = ImageDraw.Draw(slices)
    x1, y1, x2, y2 = col-10, row-10, col+10, row+10
    draw.rectangle([x1, y1, x2, y2], outline="white", width=1)
    slices.save(target_path)
    im = Image.open(target_path)

    # Create figure and axes
    fig, ax = plt.subplots()

    # Display the image
    ax.imshow(im)

def bboxOnImg(img_path, bbox):
    [x, y, height, width] = bbox
    
    x *= 640
    y *= 640
    height *= 640
    width *= 640
    
    im2 = Image.open(img_path)

    # Create figure and axes
    fig2, ax2 = plt.subplots()

    # Display the image
    ax2.imshow(im2)
    size = im2.size
    print(size)

    # Create a Rectangle patch
    rect = Rectangle((x-height/2, y - width/2), height, width, linewidth=1, edgecolor='r', facecolor='none')

    # Add the patch to the Axes
    ax2.add_patch(rect)

    plt.show()
    

   


In [ ]:
import socket
import glob
import SimpleITK as sitk
# Define the server's address and port
server_ip = '192.168.11.206'  # Listen on all available network interfaces
server_port = 16840

# Create a socket object
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Bind the socket to the address and port
server_socket.bind((server_ip, server_port))

# Listen for incoming connections
server_socket.listen(5)  # Maximum 5 queued connections

print(f"Server listening on {server_ip}:{server_port}")

# Accept incoming connection
client_socket, client_address = server_socket.accept()

print(f"Received connection from {client_address}")
for uid in candidate_dict:
    # send data
    mhd_path = glob.glob("D:\LUNA\Luna16_AugData\subset*\{}.mhd".format(uid))
    ct_mhd = sitk.ReadImage(mhd_path)
    client_socket.send(ct_mhd.encode())
    receive = b""
    while True:
        receive = client_socket.recv(1024).decode()
        if receive != None: 
            break

In [4]:
import socket

# Define the server's address and port
server_ip = '192.168.11.206'  # Listen on all available network interfaces
server_port = 16840

# Create a socket object
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Bind the socket to the address and port
server_socket.bind((server_ip, server_port))

# Listen for incoming connections
server_socket.listen(5)  # Maximum 5 queued connections

print(f"Server listening on {server_ip}:{server_port}")

# Accept incoming connection
client_socket, client_address = server_socket.accept()

print(f"Received connection from {client_address}")

# Receive and process data
data = client_socket.recv(1024)  # Receive up to 1024 bytes of data
print(f"Received data: {data.decode('utf-8')}")

# Close the sockets
client_socket.close()
server_socket.close()


Server listening on 192.168.11.206:16840
Received connection from ('192.168.11.189', 60879)
Received data: GGG


In [ ]:
s.close()
client_socket.close()
server_socket.close()

In [ ]:
# import time
# import socket
# import pickle
# s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# s.bind(('192.168.11.206', 16840))
# s.close